In [35]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("17"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Chronospatial Computer

[Open Website](https://adventofcode.com/2024/day/17)

In [36]:
import re


class Computer:
    def __init__(self, registers=None, program=None):
        if program is None:
            program = []
        if registers is None:
            registers = {}
        self.registers = registers
        self.program = program
        self.output = []
        self.pointer = 0

    def set(self, key: str, value: int):
        self.registers[key.upper()] = value

    def get(self, key) -> int:
        return self.registers[key]

    def increment(self):
        self.pointer += 2

    def get_opcode(self) -> int:
        return self.program[self.pointer]

    def get_operand(self) -> int:
        return self.program[self.pointer + 1]

    def get_combo(self) -> int:
        operand = self.get_operand()
        if operand < 4:
            return operand
        elif operand == 4:
            return self.get("A")
        elif operand == 5:
            return self.get("B")
        elif operand == 6:
            return self.get("C")
        else:
            raise Exception(f"Unknown instruction {operand}")

    def is_done(self):
        return self.pointer >= len(self.program)

    def __iter__(self):
        return iter(self.program)

    def __str__(self):
        return f"Computer: {self.registers}, {self.program}"

    def result(self):
        output_str = [str(x) for x in self.output]
        return ",".join(output_str)


# helper functions
def domain_from_input(input: InputReader) -> Computer:
    lines = input.lines_as_strs()

    computer = Computer()
    regex = re.compile(r"(\w+)")

    reading_registers = True
    for line in lines:
        if reading_registers:
            if len(line) == 0:
                reading_registers = False
                continue
            # parse registers
            key = regex.findall(line[1])[0]
            # remove :
            value = int(line[2])
            computer.set(key, value)
        else:
            for number in line[1].split(","):
                computer.program.append(int(number))

    return computer


test_input = domain_from_input(puzzle.example(0))
print(test_input)

Computer: {'A': 729, 'B': 0, 'C': 0}, [0, 1, 5, 4, 3, 0]


In [37]:
# test case (part 1)

def execute(computer: Computer):
    def perform_division(register: str, operand: int):
        numerator = computer.get("A")
        denominator = 2 ** operand
        computer.set(register, numerator // denominator)

    # calculate opcode
    opcode = computer.get_opcode()
    operand = computer.get_operand()

    # perform operation
    if opcode == 0:  # adv
        perform_division("A", computer.get_combo())
        computer.increment()
    elif opcode == 1:  # bxl
        computer.set("B", computer.get("B") ^ operand)
        computer.increment()
    elif opcode == 2:  # bst
        computer.set("B", computer.get_combo() % 8)
        computer.increment()
    elif opcode == 3:  # jnz
        if computer.get("A") != 0:
            computer.pointer = operand
        else:
            computer.increment()
    elif opcode == 4:  # bxc
        computer.set("B", computer.get("B") ^ computer.get("C"))
        computer.increment()
    elif opcode == 5:  # out
        computer.output.append(computer.get_combo() % 8)
        computer.increment()
    elif opcode == 6:  # bdv
        perform_division("B", computer.get_combo())
        computer.increment()
    elif opcode == 7:  # cdv
        perform_division("C", computer.get_combo())
        computer.increment()


def run_program(a, b, c, instructions) -> Computer:
    computer = Computer({"A": a, "B": b, "C": c}, instructions)
    while not computer.is_done():
        execute(computer)
    return computer


def part_1(reader: InputReader, debug: bool) -> str:
    computer = domain_from_input(reader)
    if debug:
        print(computer)

    while not computer.is_done():
        execute(computer)

    return computer.result()


result = run_program(0, 0, 9, [2, 6])
print(result.get("B"))
assert result.get("B") == 1

result = run_program(10, 0, 0, [5, 0, 5, 1, 5, 4])
print(result.result())
assert result.result() == "0,1,2"

result = run_program(2024, 0, 0, [0, 1, 5, 4, 3, 0])
print(result.result())
assert result.result() == "4,2,5,6,7,7,7,7,3,1,0"
print(result.get("A"))
assert result.get("A") == 0

result = run_program(0, 29, 0, [1, 7])
print(result.get("B"))
assert result.get("B") == 26

result = run_program(0, 2024, 43690, [4, 0])
print(result.get("B"))
assert result.get("B") == 44354

result = part_1(puzzle.example(0), True)
display(result)
assert result == "4,6,3,5,6,3,5,2,1,0"

1
0,1,2
4,2,5,6,7,7,7,7,3,1,0
0
26
44354
Computer: {'A': 729, 'B': 0, 'C': 0}, [0, 1, 5, 4, 3, 0]


'4,6,3,5,6,3,5,2,1,0'

In [38]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)
assert result == "3,6,7,0,5,7,3,1,4"

'3,6,7,0,5,7,3,1,4'

In [44]:
# test case (part 2)

def part_2(reader: InputReader, debug: bool) -> int:
    def run_with_a(new_value):
        computer = domain_from_input(reader)
        input = computer.program
        computer.set("A", new_value)

        while not computer.is_done():
            execute(computer)

        return input, computer.output

    original_computer = domain_from_input(reader)
    program = original_computer.program
    candidates = [0]

    # eventually the program will output the same as the end of the program
    for length in range(1, len(program) + 1):
        out = []

        # try all candidates
        for next_candidate in candidates:
            # start with 8 * num
            start = 8 * next_candidate
            # try all 3-bit combinations
            for offset in range(8):
                a = start + offset
                # run program with a
                _, output = run_with_a(a)
                # get the end of the program
                end = program[-length:]
                # if output matches the end of the program
                if output == end:
                    out.append(a)

        # the output that matches the end of the program, are the new candidates
        candidates = out

    # the smallest candidate from the last iteration is the answer
    return min(candidates)


result = part_2(puzzle.get_code_block(1), True)
print(result)
assert result == 117440

117440


In [40]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)
assert result == 164278496489149

164278496489149

In [41]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="The instruction does this using a little trampoline."><em>jumps</em></span> (The instruction does this using a little trampoline.)